In [2]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter
from collections import defaultdict

In [3]:
raw_data = []
for file in glob.glob('fhir/onlychf/*'):
    raw_data.append(pd.read_json(file))

In [18]:
def dataPipeTuples(raw_data):
    training_data = {}
    for patient in raw_data:
        id = patient['entry'][0]['resource']['id']
        training_data[id] = {}
        #total number of times patient is inpatient for chf
        training_data[id]['total_chf'] = 0
        #first time of discharge for chf and second time admitted for chf
        training_data[id]['chf_dates'] = ()
        
        for entry in patient['entry']:
            resource_type = entry['resource']['resourceType']
            if resource_type == "Encounter":
                class_code = entry['resource']['class']['code']
                if class_code == "IMP" or class_code == "EMER":
                    try:
                        reason_code = entry['resource']['reasonCode'][0]['coding'][0]['code']
                    except:
                        continue
                    start_date = entry['resource']['period']['start'][0:10]
                    if (training_data[id]['total_chf'] == 1):
                        if (pd.date_range(training_data[id]['chf_dates'][0], start_date).shape[0] > 29):
                            if reason_code == "88805009":
                                training_data[id]['total_chf'] += 1
                    else:
                        if reason_code == "88805009":
                            training_data[id]['total_chf'] += 1
                            
                if (training_data[id]['total_chf'] == 1):
                    training_data[id]['chf_dates'] = (entry['resource']['period']['end'][0:10],)
                elif (training_data[id]['total_chf'] == 2):
                    training_data[id]['chf_dates'] = (training_data[id]['chf_dates'][0], start_date)
        if (training_data[id]['total_chf'] == 0):
            del training_data[id]

    return training_data

In [19]:
def dataPipeMatrix(raw_data, tuple_list):
    training_data = {}
    for patient in raw_data:
        id = patient['entry'][0]['resource']['id']
        if id in tuple_list:
            training_data[id] = defaultdict(Counter)
            check_date = tuple_list[id]['chf_dates'][0]
            for entry in patient['entry']:
                resource_type = entry['resource']['resourceType']

                if resource_type == "Encounter":
                    start_date = entry['resource']['period']['start'][0:7]
                    if pd.date_range(start_date, check_date).shape[0] != 0:
                        try:
                            reason_code = entry['resource']['reasonCode'][0]['coding'][0]['code']
                        except:
                            reason_code = entry['resource']['type'][0]['coding'][0]['code']
                        training_data[id][start_date][reason_code] = 1
                elif resource_type == "Observation":
                    start_date = entry['resource']['effectiveDateTime'][0:7]
                    if pd.date_range(start_date, check_date).shape[0] != 0:
                        reason_code = entry['resource']['code']['coding'][0]['code']
                        training_data[id][start_date][reason_code] = 1 
                elif resource_type == "Procedure":
                    start_date = entry['resource']['performedPeriod']['start'][0:7]
                    if pd.date_range(start_date, check_date).shape[0] != 0:
                        reason_code = entry['resource']['code']['coding'][0]['code']
                        training_data[id][start_date][reason_code] = 1
                elif resource_type == "Condition":
                    start_date = entry['resource']['onsetDateTime'][0:7]
                    if pd.date_range(start_date, check_date).shape[0] != 0:
                        reason_code = entry['resource']['code']['coding'][0]['code']
                        training_data[id][start_date][reason_code] = 1
                elif resource_type == "Immunization":
                    start_date = entry['resource']['occurrenceDateTime'][0:7]
                    if pd.date_range(start_date, check_date).shape[0] != 0:
                        reason_code = entry['resource']['vaccineCode']['coding'][0]['code']
                        training_data[id][start_date][reason_code] = 1
                elif resource_type == "MedicationRequest":
                    start_date = entry['resource']['authoredOn'][0:7]
                    if pd.date_range(start_date, check_date).shape[0] != 0:
                        reason_code = entry['resource']['medicationCodeableConcept']['coding'][0]['code']
                        training_data[id][start_date][reason_code] = 1

    return training_data

In [20]:
def addMonth(dt):
    month = int(dt[5:])
    year = int(dt[0:4])
    if month == 12:
        return str(year+1)+"-01"
    else:
        date = str(month+1).zfill(2)
        return dt[0:5]+date
def addYear(dt, num):
    year = int(dt[0:4])
    date = str(year+num)
    return date+dt[4:]

In [23]:
def dataPipeDataFrames(raw_data):
    frames = []
    tuples = dataPipeTuples(raw_data)
    data = dataPipeMatrix(raw_data, tuples)
    for id in data:
        df = pd.DataFrame.from_dict(data[id], orient="index")
        df = df.reset_index().rename(columns={'index':'Month'}).sort_values(by='Month', ascending=False)
        two_year_range = np.arange(np.datetime64(addYear(tuples[id]['chf_dates'][0][0:7], -2)), np.datetime64(addMonth(tuples[id]['chf_dates'][0][0:7]))).astype(str).tolist()
        df = df.set_index('Month').reindex(two_year_range).sort_values(by='Month', ascending=False).reset_index().fillna(0)
        frames.append(df)
    all_columns = []
    for frame in frames:
        all_columns.extend(x for x in frame.columns.tolist() if not x in all_columns)
    final_frames = []
    for df in frames:
        cols = df.columns.tolist()
        cols.extend(x for x in all_columns if not x in cols)
        df = df.reindex(columns=sorted(cols, reverse=True), fill_value=0)
        final_frames.append(df)
    return (final_frames, tuples)

In [24]:
frames, tuples = dataPipeDataFrames(raw_data)

In [25]:
for df in frames:
    display(df)

,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,1991-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
1,1991-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
2,1991-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
3,1991-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
4,1991-03,0,0,0,0,0,0,0,0,0,...,0,0,0,1.0,0,0,0,0,0.0,0
5,1991-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
6,1991-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
7,1990-12,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
8,1990-11,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0
9,1990-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2015-03,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,1.0,0
1,2015-02,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
2,2015-01,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
3,2014-12,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
4,2014-11,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
5,2014-10,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
6,2014-09,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
7,2014-08,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
8,2014-07,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0
9,2014-06,0,0.0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0.0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2006-12,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0
1,2006-11,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0
2,2006-10,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0
3,2006-09,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0
4,2006-08,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0
5,2006-07,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0
6,2006-06,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,1.0,0
7,2006-05,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,1.0,0,0,0.0,0
8,2006-04,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0
9,2006-03,0,0,0.0,0,0,0,0,0,0,...,0,0.0,0,0,0,0.0,0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,1986-05,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
1,1986-04,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
2,1986-03,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
3,1986-02,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
4,1986-01,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
5,1985-12,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,1.0,0
6,1985-11,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
7,1985-10,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
8,1985-09,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0
9,1985-08,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0.0,0,0,0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2013-11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
1,2013-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
2,2013-09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
3,2013-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
4,2013-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
5,2013-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
6,2013-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
7,2013-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
8,2013-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
9,2013-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0


...

,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2009-01,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0
1,2008-12,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0
2,2008-11,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0
3,2008-10,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0
4,2008-09,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0
5,2008-08,0,0,0,0,0,0,0.0,0,0,...,0,1.0,0,0,0,0.0,1.0,0,0.0,0
6,2008-07,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0
7,2008-06,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,1.0,0
8,2008-05,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0
9,2008-04,0,0,0,0,0,0,0.0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2016-06,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
1,2016-05,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
2,2016-04,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
3,2016-03,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
4,2016-02,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
5,2016-01,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
6,2015-12,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
7,2015-11,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
8,2015-10,0,0,0,0,0,0,0,0,1.0,...,0,0,0,0,0,0,0,0,0.0,0
9,2015-09,0,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2012-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0
1,2011-12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
2,2011-11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
3,2011-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
4,2011-09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
5,2011-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
6,2011-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
7,2011-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
8,2011-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
9,2011-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2005-01,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
1,2004-12,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
2,2004-11,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
3,2004-10,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
4,2004-09,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
5,2004-08,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
6,2004-07,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,1.0,0,0,0.0,0
7,2004-06,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
8,2004-05,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
9,2004-04,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0


,Month,999967,996740,966222,92691004,90781000119102,90470006,904419,90226004,897718,...,106892,10509002,1049630,1049221,10480-2,104435004,1043400,10383002,10230-1,1000126
0,2019-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
1,2019-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
2,2019-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
3,2019-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
4,2019-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
5,2019-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
6,2019-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0
7,2018-12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
8,2018-11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
9,2018-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0


In [26]:
pd.DataFrame.from_dict(tuples, orient="index")

,total_chf,chf_dates
372a8aec-7a80-4a7d-807c-7a25856d714a,1,"(1991-07-06,)"
b7742521-7ed3-4439-a51f-a90f5ec4db3f,3,"(2015-03-30, 2015-09-03)"
936e8aa8-78ee-465d-80c7-36e5244af270,1,"(2006-12-12,)"
89ba0939-5119-4b80-920d-fe775c27d1c2,1,"(1986-05-31,)"
a1b2df73-406a-4d17-801b-82354322f13a,1,"(2013-11-23,)"
6aa267d0-af90-4e12-aa19-eb867008445d,1,"(2001-01-04,)"
df000049-5a72-4260-becf-71c91114c5cf,1,"(2001-12-08,)"
ba27d17d-51f3-4d46-a98b-75e374baf7a5,1,"(2018-08-11,)"
c1abb66f-3c24-4074-b79f-cde90cef687e,1,"(2017-04-09,)"
1fc839e5-6f45-4f63-a040-80fc91a5d4df,1,"(2006-09-01,)"
